In [1]:
import numpy as np
import matplotlib.pyplot as plt
from transforms3d.euler import euler2mat
from mpl_toolkits.mplot3d import Axes3D
from bvh import Bvh, BvhNode
import math
# packages for 3d interactivate visualization
import plotly
import plotly.graph_objs as go 
import copy

In [2]:
class MyNode:
    def __init__(self,name):
        self.name = name
        self.children = []
        self.parent = None
        self.parent_index = None
        self.offset = self.channels = self.index = None
        self.pos = self.matrix = None

In [3]:
def make_parent(Nodes):
    for num in range(0,len(Nodes)):
        node = Nodes[num]
        if(node.parent_index != None):
            pNum = node.parent_index
            node.parent = Nodes[pNum]
            Nodes[pNum].children.append(node)

In [4]:
def get_nodes(joints):
    Nodes = {}
    for num in range(0,len(joints)):
        name = joints[num].value[1]
        node = MyNode(name)
        node.offset = np.array(mocap.joint_offset(name))
        node.channels = mocap.joint_channels(name)
        node.index = mocap.get_joint_index(name)
        if(num > 0):
            parent_name = joints[num].parent.value[1]
            node.parent_index = mocap.get_joint_index(parent_name)
        else:
            node.parent_index = None
        Nodes[num] = node
    make_parent(Nodes)
    return Nodes

In [5]:
def get_motions(frame_idx):
    motions= []
    root_pos = mocap.frame_joint_channels(frame_idx, 'Hips', ['Xposition','Yposition','Zposition'])
    root_pos = np.array(root_pos)
    names = mocap.get_joints_names()
    for num in range(0,len(Nodes)):
        name = names[num]
        motion = mocap.frame_joint_channels(frame_idx, name, ['Xrotation','Yrotation','Zrotation'])
        motions.append(motion)
    return motions,root_pos

In [6]:
def euler_to_matrix(euler,axis = 'szyx'):
    rotation = np.deg2rad(euler)
    matrix = euler2mat(*rotation,axis)
    return matrix
def euler_to_matrix_test(euler):
    rotation = np.deg2rad(euler)
    alpha = rotation[0]
    beta = rotation[1]
    gamma = rotation[2]
    ca = math.cos(alpha)
    cb = math.cos(beta)
    cg = math.cos(gamma)
    sa = math.sin(alpha)
    sb = math.sin(beta)
    sg = math.sin(gamma)        

    Rx = np.asarray([[1, 0, 0], 
          [0, ca, sa], 
          [0, -sa, ca]
          ])

    Ry = np.asarray([[cb, 0, -sb], 
          [0, 1, 0],
          [sb, 0, cb]])

    Rz = np.asarray([[cg, sg, 0],
          [-sg, cg, 0],
          [0, 0, 1]])

    rotmat = np.eye(3)

    rotmat = np.matmul(Ry, rotmat)
    rotmat = np.matmul(Rx, rotmat)
    rotmat = np.matmul(Rz, rotmat)
    return rotmat

In [7]:
def compute_pos(Nodes,motions,root_pos):
    for num in range(0,len(Nodes)):
        node = Nodes[num]
        rot_matrix = euler_to_matrix_test(motions[num])
        if(node.parent_index != None):
            node.matrix = rot_matrix.dot(node.parent.matrix)
            direction = node.offset.dot(node.parent.matrix)
            node.pos =  node.parent.pos + direction
        else:
            node.pos =  root_pos
            node.matrix = rot_matrix

In [8]:
with open('data/bvh/0005_BackwardsWalk001.bvh.txt') as f:
        mocap = Bvh(f.read())

In [9]:
joints = mocap.get_joints()
Nodes = get_nodes(joints)
print(len(joints))
print(joints)

25
[ROOT Hips, JOINT LeftUpLeg, JOINT LeftLeg, JOINT LeftFoot, JOINT LeftToeBase, JOINT RightUpLeg, JOINT RightLeg, JOINT RightFoot, JOINT RightToeBase, JOINT Spine, JOINT Spine1, JOINT Neck, JOINT Head, JOINT LeftShoulder, JOINT LeftArm, JOINT LeftForeArm, JOINT LeftHand, JOINT LeftHandThumb, JOINT L_Wrist_End, JOINT RightShoulder, JOINT RightArm, JOINT RightForeArm, JOINT RightHand, JOINT RightHandThumb, JOINT R_Wrist_End]


In [10]:
framenum = mocap.nframes
print(framenum)
print(len(Nodes))

1239
25


In [11]:
Nodes_lst = []
for frame_idx in range(0,framenum//2,20):
    motions,root_pos = get_motions(frame_idx)
    compute_pos(Nodes,motions,root_pos)
    Nodes_lst.append(copy.deepcopy(Nodes))

In [12]:
# for Nodes in Nodes_lst[:5]:
#     for node in Nodes.values():
#         if(node.name == "LeftShoulder" 
#            or node.name == "RightShoulder"
#            or node.name == "Neck"):
#             print(node.pos,node.offset)

In [14]:
for (idx,node) in Nodes_lst[20].items():
    print(idx,(node.index,node.name))

0 (0, 'Hips')
1 (1, 'LeftUpLeg')
2 (2, 'LeftLeg')
3 (3, 'LeftFoot')
4 (4, 'LeftToeBase')
5 (5, 'RightUpLeg')
6 (6, 'RightLeg')
7 (7, 'RightFoot')
8 (8, 'RightToeBase')
9 (9, 'Spine')
10 (10, 'Spine1')
11 (11, 'Neck')
12 (12, 'Head')
13 (13, 'LeftShoulder')
14 (14, 'LeftArm')
15 (15, 'LeftForeArm')
16 (16, 'LeftHand')
17 (17, 'LeftHandThumb')
18 (18, 'L_Wrist_End')
19 (19, 'RightShoulder')
20 (20, 'RightArm')
21 (21, 'RightForeArm')
22 (22, 'RightHand')
23 (23, 'RightHandThumb')
24 (24, 'R_Wrist_End')
